# Solution computation from models

This notebook demonstrates the calculation of a solution to a model using multiple rays:
- Creating Moon models and mesh (M1, M2, M3, weber_core_smooth, weber_core)
- Source and receiver definition using fibonacci equidistant spacings at given radii
- Computation of travel times
- Inversion of data using LU factorization

In [ ]:
import numpy as np
from sensray import PlanetModel
from fwdop import GFwdOp, make_scalar_field
from pygeoinf.linear_solvers import LUSolver, CholeskySolver
from ray_and_point_generation import get_rays, fibonacci_sphere_points
from itertools import product
from random import randint

## 1. Creation of Moon model and mesh

Creating a model and mesh for ray tracing

### Creating model

Creating a moon model from information stored in .nd file
- User-defined model (model_name), mesh size, and inner core mesh size
- Model creation of PlanetModel class

In [29]:
# Load model and create mesh
model_name = "M1"
mesh_size_km = 1000

# Create mesh and save if not exist, otherwise load existing
mesh_path = "M1_mesh"

# Load model and create mesh
model = PlanetModel.from_standard_model('M1')

### Creating layered tetrahedral mesh

Uses a layered tetrahedral mesh to control resolution across discontinuities
- User-defined mesh name
- Radii and H_layers gathered from the model properties
- Populate mesh properties with vp, vs, rho from the file (also gathered from model)

In [30]:
try:
    model.create_mesh(from_file=mesh_path)
    print(f"Loaded existing mesh from {mesh_path}")
except FileNotFoundError:
    print("Creating new mesh...")
    radii = model.get_discontinuities()
    H_layers = [1000, 600]
    model.create_mesh(mesh_size_km=mesh_size_km, radii=radii, H_layers=H_layers)
    model.mesh.populate_properties(model.get_info()["properties"])
    model.mesh.save(f"{model_name}_mesh")  # Save mesh to VT
print(f"Created mesh: {model.mesh.mesh.n_cells} cells")

Loaded mesh from M1_mesh.vtu
Loaded metadata: 9975 cells, 2258 points
Loaded existing mesh from M1_mesh
Created mesh: 9975 cells


## 2. Source-Receiver geometry and travel time computation

Set up equally spaced sources at the surface (depth 0) and receivers at a constant depth (single depth within range)

In [ ]:
# Generate sources and receivers

setup_info = {
    "source": {"N": 5, "min depth": 150, "max depth": 150},
    "receiver": {"N": 5, "min depth": 0, "max depth": 0},
}

# Example usage:
# get evenly distributed points at a single depth
print("Generate equally spaced sources (single depth) and receivers (surface)")
depth = randint(setup_info["source"]["min depth"], setup_info["source"]["max depth"])
sources = fibonacci_sphere_points(5, radius=model.radius-depth, latlon=True)  # 20 sources at 150km depth
receivers = fibonacci_sphere_points(5, radius=model.radius, latlon=True)  # 20 stations on Earth radius
phases = ["P"]
print(sources)
print(receivers)

print("Compute ray for each source-receiver-phase combination...")
srr = get_rays(model=model, srp=product(sources, receivers, phases), radius=True)

Compute ray for each source-receiver-phase combination...
Building obspy.taup model for '/home/matth/Masters-Project/SensRay/sensray/models/M1.nd' ...
filename = /home/matth/Masters-Project/SensRay/sensray/models/M1.nd
Done reading velocity model.
Radius of model . is 1737.1
Using parameters provided in TauP_config.ini (or defaults if not) to call SlownessModel...
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 kilometers
taup.create.max_range_interval = 0.04363323129985824 degrees
taup.create.max_interp_error = 0.05 seconds
taup.create.allow_inner_core_s = True
Slow model  643 P layers,747 S layers
Done calculating Tau branches.
Done Saving /tmp/M1.npz
Method run is done, but not necessarily successful.


### Calculate travel times

Compute sensitivity kernels and project functions on the mesh

Calculate travel times for the function as in other demos

In [ ]:
G = GFwdOp(model=model, rays=srr[:,2])

# Generate different models and calculate dv
functions = {
    "simple": {"R": lambda r: np.ones_like(r), "T": lambda theta, phi: np.ones_like(theta)},
    "complex": {"R": lambda r: r**2 * np.exp(-r/100000), "T": lambda theta, phi: np.cos(theta)},
    "harmonic": {"R": lambda r: 0.1 * model.get_property_at_radius(radius=r, property_name="vp"), "T": lambda theta, phi: 0.5 * np.sqrt(5 / np.pi) * (3 * np.cos(theta) ** 2 - 1)},
}

func = "harmonic"
f = make_scalar_field(functions[func]["R"], functions[func]["T"])

model.mesh.project_function_on_mesh(f, property_name="dv")
print("Cell data 'dv':", model.mesh.mesh.cell_data["dv"])

travel_times = G(model.mesh.mesh.cell_data["dv"])
print(travel_times)

Calculate sensitivity kernels and sparse G matrix...
Stored sensitivity kernel as cell data: 'K_P_vp'
P Kernel Matrix shape: (1, 9975), nnz: 24
Scalar field generation...
Dv projection onto mesh...
Time travel computation...
[8.34158499]


## 3. Compute inverse operator

- Compute inverse solution M_tilde and add to model
- Display slice to visualise solution


### Compute cell inverse and M_tilde

Compute solution M_tilde (vector of length Nd) from the cell inverse applied to the travel times

Add the solution to the cell data

In [ ]:
print("Compute  cell inverse and add to model mesh...")
M_tilde = (G.adjoint@solver(G@G.adjoint))(travel_times)
model.mesh.mesh.cell_data["solution"] = M_tilde

True


### Visualise solution

Plot the solution as a slice through the sphere on the (0, 1, 0) plane normal

In [ ]:
print("Solution visualization...")
plotter1 = model.mesh.plot_cross_section(plane_normal=(0, 1, 0), property_name="solution")

plotter1.camera.position = (8000, 6000, 10000)

plotter1.show()